In [ ]:
 %%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

TIMESTEP=50e-6
DURATION=0.2

EMT_Slack_PiLine_PQLoad_with_PF_Init --timestep=${TIMESTEP} --duration=${DURATION}
DP_Slack_PiLine_PQLoad_with_PF_Init --timestep=${TIMESTEP} --duration=${DURATION}

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import re

model_name = 'Slack_PiLine_PQLoad_with_PF_Init'

path_DP = 'logs/' + 'DP_' + model_name + '_DP/'
dpsim_result_file_DP = path_DP  + 'DP_' + model_name + '_DP.csv'
ts_dpsim_DP = read_timeseries_csv(dpsim_result_file_DP)

path_EMT = 'logs/' + 'EMT_' + model_name + '_EMT/'
dpsim_result_file_EMT = path_EMT  + 'EMT_' + model_name + '_EMT.csv'
ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file_EMT)

## Plot voltages

In [ ]:
import matplotlib.pyplot as plt

%matplotlib widget

plt.figure(figsize=(12,8))

var_names = ['v1','v2']
for var_name in var_names:   
    plt.plot(ts_dpsim_EMT[var_name+'_0'].interpolate(50e-6).time, np.sqrt(3/2)*ts_dpsim_EMT[var_name+'_0'].interpolate(50e-6).values, label='EMT')
    plt.plot(ts_dpsim_DP[var_name].interpolate(50e-6).time, ts_dpsim_DP[var_name].interpolate(50e-6).frequency_shift(50).values, label='DP backshift', linestyle='--')
    
plt.legend()
plt.show()

## Plot current

In [ ]:
var_name = 'i12'

ts_emt_compare = TimeSeries('ts_emt_compare', ts_dpsim_EMT[var_name+'_0'].interpolate(50e-6).time, np.sqrt(3/2)*ts_dpsim_EMT[var_name+'_0'].interpolate(50e-6).values)
ts_dp_compare = TimeSeries('ts_dp_compare', ts_dpsim_DP[var_name].interpolate(50e-6).time, ts_dpsim_DP[var_name].interpolate(50e-6).frequency_shift(50).values)

plt.figure(figsize=(12,8))
plt.plot(ts_emt_compare.time, ts_emt_compare.values, label='EMT')
plt.plot(ts_dp_compare.time, ts_dp_compare.values, label='DP backshift', linestyle='--')
plt.legend()
plt.show()

## Calculate difference

In [ ]:
ts_emt_compare.rmse(ts_emt_compare, ts_dp_compare)